In [1]:
import numpy as np
import scipy
import scipy.io as sio
from pyriemann.utils.mean import mean_covariance
import sklearn.datasets
import sklearn.decomposition
from scipy.spatial import distance

np.seterr(divide='ignore', invalid='ignore')



{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:

def get_data():
    '''
    Navigates through file tree and extracts FCs with optional reconstruction
    '''
    # Yeo ordering
    fname = '../data/100_unrelated.csv'
    yeo = True
    if yeo:
        yeo_order = list(sio.loadmat("../data/yeo_RS7_N374.mat",
                                     squeeze_me=True,
                                     struct_as_record=False)['yeoOrder'] - 1)
    # Load subject ID and task names
    subjectids = np.loadtxt(fname, dtype=np.int)
    nSubj = len(subjectids)
    tasks = ['rfMRI_REST1_LR', 'rfMRI_REST1_RL', 'rfMRI_REST2_LR',
             'rfMRI_REST2_RL', 'tfMRI_EMOTION_LR', 'tfMRI_EMOTION_RL',
             'tfMRI_GAMBLING_LR', 'tfMRI_GAMBLING_RL', 'tfMRI_LANGUAGE_LR',
             'tfMRI_LANGUAGE_RL', 'tfMRI_MOTOR_LR', 'tfMRI_MOTOR_RL',
             'tfMRI_RELATIONAL_LR', 'tfMRI_RELATIONAL_RL', 'tfMRI_SOCIAL_LR',
             'tfMRI_SOCIAL_RL', 'tfMRI_WM_LR', 'tfMRI_WM_RL']
    M = {}
    # Walk through file tree and extract FCs
    for task in tasks:
        masterFC_dir = '../data/results_SIFT2'
        restingstatename = 'fMRI/' + task + '/FC/FC_glasser_subc_GS_bp_z.mat'
        task_matrices = []
        for subject in subjectids:
            filename = masterFC_dir + '/' + \
                str(subject) + '/' + restingstatename
            mat = sio.loadmat(filename, squeeze_me=True,
                              struct_as_record=False)
            A_orig = mat['FC']
            if yeo:
                A_orig = A_orig[np.ix_(yeo_order, yeo_order)]
            np.fill_diagonal(A_orig, 1)
            task_matrices.append(A_orig)
        M[task] = np.array(task_matrices)
    test = np.concatenate((M['rfMRI_REST1_LR'], M['tfMRI_EMOTION_LR'],
                           M['tfMRI_GAMBLING_LR'], M['tfMRI_LANGUAGE_LR'],
                           M['tfMRI_MOTOR_LR'], M['tfMRI_RELATIONAL_LR'],
                           M['tfMRI_SOCIAL_LR'], M['tfMRI_WM_LR']))
    retest = np.concatenate((M['rfMRI_REST1_RL'], M['tfMRI_EMOTION_RL'],
                             M['tfMRI_GAMBLING_RL'], M['tfMRI_LANGUAGE_RL'],
                             M['tfMRI_MOTOR_RL'], M['tfMRI_RELATIONAL_RL'],
                             M['tfMRI_SOCIAL_RL'], M['tfMRI_WM_RL']))
    del M
    all_FC = np.concatenate((test, retest))
    del test, retest
    return all_FC, nSubj


def q1invm(q1, eig_thresh=0):
    U, S, V = scipy.linalg.svd(q1)
    s = np.diag(S)
    s[s < eig_thresh] = eig_thresh
    S = np.diag(s ** (-1 / 2))
    Q1_inv_sqrt = U * S * np.transpose(V)
    Q1_inv_sqrt = (Q1_inv_sqrt + np.transpose(Q1_inv_sqrt)) / 2
    return Q1_inv_sqrt


def qlog(q):
    U, S, V = scipy.linalg.svd(q)
    s = np.diag(S)
    S = np.diag(np.log(s))
    Q = U * S * np.transpose(V)
    return Q


def tangential(all_FC, ref):
    # Regularization for riemann
    if ref in ['riemann', 'kullback_sym', 'logeuclid']:
        print("Adding regularization!")
        eye_mat = np.eye(all_FC.shape[1])
        scaling_mat = np.repeat(eye_mat[None, ...], all_FC.shape[0], axis=0)
        all_FC += scaling_mat
    Cg = mean_covariance(all_FC, metric=ref)
    Q1_inv_sqrt = q1invm(Cg)
    Q = Q1_inv_sqrt @ all_FC @ Q1_inv_sqrt
    tangent_FC = np.array([qlog(a) for a in Q])
    return tangent_FC


def pca_recon(FC, pctComp=None):
    '''
    Reconstructs FC based on number of principle components
    '''
    if pctComp is None:
        return FC
    FC = np.reshape(FC, (FC.shape[0], -1))
    nComp = int(FC.shape[0] * pctComp)
    mu = np.mean(FC, axis=0)
    pca_rest = sklearn.decomposition.PCA()
    pca_rest.fit(FC)
    SCORES = pca_rest.transform(FC)[:, :nComp]
    COEFFS = pca_rest.components_[:nComp, :]
    FC_recon = np.dot(SCORES, COEFFS)
    del SCORES, COEFFS
    FC_recon += mu
    FC_recon = np.reshape(FC_recon, (FC.shape[0], 374, 374))
    return FC_recon


   

In [74]:
ref = 'euclid' 
distance_method = input("Which distance metric? ").lower()
# Navigate tree and get raw correlation FC matrices
print("Importing all correlation matrices...", end=" ")
all_FC, nSubj = get_data()
print("All FCs successfully loaded!\n")

Which distance metric? corr
Importing all correlation matrices... All FCs successfully loaded!



In [94]:
labels = np.tile(np.repeat(np.arange(0, 8), nSubj), 2)
indices = np.random.permutation(all_FC.shape[0])
train_idx = indices[:int(0.80 * all_FC.shape[0])]
test_idx = indices[int(0.80 * all_FC.shape[0]):]

In [102]:
for ref in ['none']:
    all_FC, nSubj = get_data()
    if ref != 'none':
        all_FC = tangential(all_FC, ref)
    train_FCs = np.zeros((len(train_idx),70125), dtype=np.float32)
    for idx, mat in enumerate(all_FC[train_idx]):
        train_FCs[idx] = mat[np.triu_indices(mat.shape[0], k=0)]
    test_FCs = np.zeros((len(test_idx),70125), dtype=np.float32)
    for idx, mat in enumerate(all_FC[test_idx]):
        test_FCs[idx] = mat[np.triu_indices(mat.shape[0], k=0)]
        
    for distance_method in ['corr', 'cosine', 'euclidean']:
        print(f'Testing {ref} and {distance_method}...')
        num_correct = 0
        for idx1, mat1 in enumerate(test_FCs):
            min_dist = np.inf
            true_label = labels[test_idx[idx1]]
            for idx2, mat2 in enumerate(train_FCs):
                if distance_method == 'corr':
                    temp_dist = distance.correlation(mat1, mat2)
                elif distance_method == 'cosine':
                    temp_dist = distance.cosine(mat1, mat2)
                else:
                    temp_dist = distance.euclidean(mat1, mat2)
                if temp_dist < min_dist:
                    min_dist = temp_dist
                    best_idx = train_idx[idx2]
            pred_label = labels[best_idx]
            if pred_label == true_label:
                num_correct += 1
        accuracy = num_correct / len(test_idx)
        accuracies[ref+"_"+distance_method] = accuracy

Testing none and corr...
Testing none and cosine...
Testing none and euclidean...


In [103]:
accuracies

{'euclid_corr': 0.881578947368421,
 'euclid_cosine': 0.8651315789473685,
 'euclid_euclidean': 0.14802631578947367,
 'harmonic_corr': 0.8519736842105263,
 'harmonic_cosine': 0.8914473684210527,
 'harmonic_euclidean': 0.13815789473684212,
 'logeuclid_corr': 0.20394736842105263,
 'logeuclid_cosine': 0.19736842105263158,
 'logeuclid_euclidean': 0.20723684210526316,
 'kullback_sym_corr': 0.18421052631578946,
 'kullback_sym_cosine': 0.1875,
 'kullback_sym_euclidean': 0.20394736842105263,
 'riemann_corr': 0.19736842105263158,
 'riemann_cosine': 0.20394736842105263,
 'riemann_euclidean': 0.19736842105263158,
 'none_corr': 0.5098684210526315,
 'none_cosine': 0.5032894736842105,
 'none_euclidean': 0.5164473684210527}

In [104]:
import csv

a_file = open("../results/distances.csv", "w")

writer = csv.writer(a_file)
for key, value in accuracies.items():
    writer.writerow([key, value])
    
a_file.close()